## Preparations

## Tham khảo


*   https://github.com/clam004/RL-Chat-pytorch
*   https://github.com/GameDisplayer/DRL4DG
*   https://pytorch.org/tutorials/beginner/chatbot_tutorial.html


In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math
import numpy as np
from scipy.spatial import distance


USE_CUDA = torch.cuda.is_available()
device = torch.device("cuda" if USE_CUDA else "cpu")
#device = torch.device("cpu")

## fix lỗi tensor 

In [ ]:
!pip install torch==1.6.0 torchvision==0.7.0

     |████████████████████████████████| 748.8MB 17kB/s 
     |████████████████████████████████| 5.9MB 42.1MB/s 
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.6.0 which is incompatible.
  Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
  Found existing installation: torchvision 0.10.0+cu102
    Uninstalling torchvision-0.10.0+cu102:
      Successfully uninstalled torchvision-0.10.0+cu102


## Load data from drive

In [ ]:

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Tạo Folder CS106 và clone data từ link git sau [data](https://github.com/GameDisplayer/DRL4DG/tree/main/chatbot)

In [ ]:
cd /content/drive/MyDrive/CS106

/content/drive/MyDrive/CS106


## Load & Preprocess Data 

Let's have a look to our dialydialog dataset : https://www.aclweb.org/anthology/I17-1099/

In [ ]:
corpus_name = "dailydialog"
corpus = os.path.join("data", corpus_name)

def printLines(file, n=10):
    with open(file, 'r', encoding="utf-8") as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)

printLines(os.path.join(corpus, "dialogues_text.txt"))

The kitchen stinks . __eou__ I'll throw out the garbage . __eou__

So Dick , how about getting some coffee for tonight ? __eou__ Coffee ? I don ’ t honestly like that kind of stuff . __eou__ Come on , you can at least try a little , besides your cigarette . __eou__ What ’ s wrong with that ? Cigarette is the thing I go crazy for . __eou__ Not for me , Dick . __eou__

Are things still going badly with your houseguest ? __eou__ Getting worse . Now he ’ s eating me out of house and home . I ’ Ve tried talking to him but it all goes in one ear and out the other . He makes himself at home , which is fine . But what really gets me is that yesterday he walked into the living room in the raw and I had company over ! That was the last straw . __eou__ Leo , I really think you ’ re beating around the bush with this guy . I know he used to be your best friend in college , but I really think it ’ s time to lay down the law . __eou__ You ’ re right . Everything is probably going to come to a head to

## Create formatted data file

We'll create a formatted data file in which each line contains a tab-separated query sentence and a response sentence pair.

The following functions pase the `dailogues_text.txt` data file.
- `loadLines` splits each line of the file into conversations
- `extractSentencePairs` extracts pairs of sentences from conversations

In [ ]:
# Splits each line of the file into a dictionary of fields
def loadLines(fileName):
    conversations = []
    with open(fileName, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split("__eou__")
            conversations.append(values)
    return conversations


# Extracts pairs of sentences from conversations
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        # Iterate over all the lines of the conversation
        for i in range(len(conversation) - 1):  # We ignore the last line (no answer for it)
            inputLine = conversation[i].strip()
            targetLine = conversation[i+1].strip()
            # Filter wrong samples (if one of the lists is empty)
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs

Now we call the above functions to create a new file : `formatted_dialogues_text.txt`

In [ ]:
# Define path to new file
datafile = os.path.join(corpus, "formatted_dialogues_text.txt")

delimiter = '\t'
# Unescape the delimiter
delimiter = str(codecs.decode(delimiter, "unicode_escape"))

print("\nLoading conversations...")
conversations = loadLines(os.path.join(corpus, "dialogues_text.txt"))

# Write new csv file
print("\nWriting newly formatted file...")
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)

# Print a sample of lines
print("\nSample lines from file:")
printLines(datafile)


Loading conversations...

Writing newly formatted file...

Sample lines from file:
The kitchen stinks .	I'll throw out the garbage .

So Dick , how about getting some coffee for tonight ?	Coffee ? I don ’ t honestly like that kind of stuff .

Coffee ? I don ’ t honestly like that kind of stuff .	Come on , you can at least try a little , besides your cigarette .

Come on , you can at least try a little , besides your cigarette .	What ’ s wrong with that ? Cigarette is the thing I go crazy for .

What ’ s wrong with that ? Cigarette is the thing I go crazy for .	Not for me , Dick .

Are things still going badly with your houseguest ?	Getting worse . Now he ’ s eating me out of house and home . I ’ Ve tried talking to him but it all goes in one ear and out the other . He makes himself at home , which is fine . But what really gets me is that yesterday he walked into the living room in the raw and I had company over ! That was the last straw .

Getting worse . Now he ’ s eating me out of 

## Load and trim data

Now let's create a vocabulary and load query/response sentence pairs into memory.

First we must create a mapping of each word to a discrete numerical space (the index value).

Voc class keeps the mapping from words to indexes, a reverse mapping of indexes to words, a count of each word and a total word count.
There are 3 central methods :
- `addWord` to add a word to the vocabulary
- `addSentence` to add all words in a sentence
- `trim` for trimming infrequently seen words

In [ ]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True

        keep_words = []

        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))

        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

Before assemble our vocabulary and query/response sentence pairs we must perform some preprocessing.

1. Convert the Unicode strings to ASCII with `unicodeToAscii`.
2. Convert all letters to lowercase and trim all non-letter characters except basic punctuation `normalizeString`
3. Filter out sentences witg length greater than the `MAX_LENGTH` threshold in `filterPairs`


In [ ]:
#handle dull_responses now
dull_responses = ["I do not know what you are talking about.", "I do not know.", 
 "You do not know.", "You know what I mean.", "I know what you mean.", 
 "You know what I am saying.", "You do not know anything."]

In [ ]:
MAX_LENGTH = 15  # Maximum sentence length to consider

# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# Read query/response pairs and return a voc object
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # Read the file and split into lines
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# Returns True iff both sentences in a pair 'p' are under the MAX_LENGTH threshold
def filterPair(p):
    # Input sequences need to preserve the last word for EOS token
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# Filter pairs using filterPair condition
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]


# Using the functions defined above, return a populated voc object and pairs list
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data ...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for d in dull_responses:
        voc.addSentence(d)
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs

Finally assmble voc and pairs

In [ ]:
# Load/Assemble voc and pairs
save_dir = os.path.join("data", "save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
# Print some pairs to validate
print("\npairs:")
for pair in pairs[:10]:
    print(pair)

Start preparing training data ...
Reading lines...
Read 89862 sentence pairs
Trimmed to 43724 sentence pairs
Counting words...
Counted words: 10126

pairs:
['the kitchen stinks .', 'i ll throw out the garbage .']
['so dick how about getting some coffee for tonight ?', 'coffee ? i don t honestly like that kind of stuff .']
['coffee ? i don t honestly like that kind of stuff .', 'come on you can at least try a little besides your cigarette .']
['would you mind waiting a while ?', 'well how long will it be ?']
['i swear i m going to kill you for this .', 'what s wrong ? didn t you think it was fun ? !']
['never ! but thank you for inviting me .', 'come on . you ll feel better after we hit the showers .']
['certainly . how about spaghetti with clams and shrimps .', 'sounds delicious . ok . she ll try that .']
['can you manage chopsticks ?', 'why not ? see .']
['why not ? see .', 'good mastery . how do you like our chinese food ?']
['i m exhausted .', 'okay let s go home .']


### Trimming rarely used words out of vocab

One tactic beneficial to achieve faster convergence during training is trimming rarely used words out of our vocabulary.

1. Trim words used under `MIN_COUNT` threshold using `voc.trim`
2. Filter out pairs with trimmed words

In [ ]:
MIN_COUNT = 3    # Minimum word count threshold for trimming

def trimRareWords(voc, pairs, MIN_COUNT):
    # Trim words used under the MIN_COUNT from the voc
    voc.trim(MIN_COUNT)
    # Filter out pairs with trimmed words
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # Check input sentence
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # Check output sentence
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break

        # Only keep pairs that do not contain trimmed word(s) in their input or output sentence
        if keep_input and keep_output:
            keep_pairs.append(pair)

    print("Trimmed from {} pairs to {}, {:.4f} of total".format(len(pairs), len(keep_pairs), len(keep_pairs) / len(pairs)))
    return keep_pairs


# Trim voc and pairs
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 6159 / 10123 = 0.6084
Trimmed from 43724 pairs to 38716, 0.8855 of total


## Prepare Data for Models

BATCH TECHNIQUE

To accomodate sentences of different sizes in the same batch we make our batched input tensor of shape `(max_length, batch_size)` where sentences shorter than the max_length are zeropadded after the `EOS_token`.

- `inputVar` function handles the process of converting sentences to tensor. It returns a tensor of `lengths` for each sequence in the batch for the decoder.
- `outputVar` function performs the same as `inputVar` but returns a binary mask tensor and a maximum target sentence length.
- `batch2TrainData` takes a bunch of pairs and returns the input and target tensors

In [ ]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


def zeroPadding(l, fillvalue=PAD_token):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

def binaryMatrix(l, value=PAD_token):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[ 130,    7,   50,  811,  187],
        [  15, 1727,    8,   16,   15],
        [  74,   37,   34,   17,    7],
        [   8, 1753,   26,   58,  159],
        [  15,   99,   27,  183,   57],
        [ 154,    4,  239,  245,   30],
        [  16,    8,   30,   71,    2],
        [  95,   34,    2,    2,    0],
        [  48,  486,    0,    0,    0],
        [2653,   30,    0,    0,    0],
        [ 111,    2,    0,    0,    0],
        [  30,    0,    0,    0,    0],
        [   2,    0,    0,    0,    0]])
lengths: tensor([13, 11,  8,  8,  7])
target_variable: tensor([[ 156,   16,  130,  176,  145],
        [   8,   34,   74,    9,    8],
        [   9, 5403,    8,    8,  348],
        [  15,  164,   15,   61,  184],
        [   2,  374,    2,   30,   62],
        [   0,   15,    0,    2,   13],
        [   0,  107,    0,    0,  177],
        [   0,    8,    0,    0,   30],
        [   0,  826,    0,    0,   80],
        [   0,   98,    0,    0,  139],
        

## Seq2Seq

The brain of our chatbot is a sequence to sequence model.

Một RNN hoạt động như một _encoder_ mã hóa chuỗi đầu vào có độ dài thay đổi thành vectơ ngữ cảnh có độ dài cố định (lớp ẩn cuối cùng của RNN).
RNN thứ hai là _decoder_ nhận đầu vào một từ và một vectơ ngữ cảnh và trả về một dự đoán cho từ tiếp theo trong chuỗi.

 ### Encoder



```
# This is formatted as code
```

Bộ mã hóa RNN lặp qua các mã thông báo và xuất ra một vectơ "đầu ra" và một vectơ "sate ẩn". vectơ trạng thái ẩn được chuyển sang bước thời gian tiếp theo trong khi vectơ đầu ra là bộ ghi.
Bộ mã hóa chuyển đổi bối cảnh mà nó nhìn thấy tại mỗi điểm trong chuỗi thành một tập hợp các điểm trong không gian chiều cao. Bộ giải mã sẽ sử dụng nó để tạo ra từ đầu ra.

Chúng tôi sử dụng Thiết bị định kỳ lặp lại nhiều lớp hai chiều.
Nó mang lại lợi thế cho việc mã hóa cả ngữ cảnh trong quá khứ và tương lai!

Computation Graph :
1. Convert word indexed to embeddings
2. Pack padded batch of sequences for RNN module
3. Forward pass through GRU
4. Unpack padding
5. Sum bidirectional GRU outputs
6. Return output and final hidden state

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

### Decoder

The decoder RNN uses the encoder's context vectors and internal hidden states to generate the next word of the sequence.
It continues generating words until an `EOS_token`.
The problem with a vanilla seq2seq decoder is that if we rely woley on the context vector it will have information loss. (especially with long input sequences).

-> `attention mechanism` allows the decoder to pay attention to certain parts of the input sequence.

In [ ]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

Now that the attention submodule is implemented let's dive into the actual decoder model.

Computation Graph:
1. Get embedding of current input word.
2. Forward through unidirectional GRU
3. Calculate attention weights from the current GRU output
4. Multiply attention weights to encoder outputs to get a new context vector
5. Concatenate weighted context vector and GRU using Luong eq
6. Predict next word
7. Return output and final hidden state

In [ ]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

## Training 

`maskNNLLLoss` calculates the average negative log likelihood of the elements that correspond to a 1 in the mask tensor.

In [ ]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

Single training iteration (single batch of inputs)

Couple of clever tricks :
- `teacher forcing` at some probability (set by `teacher_forcing_ratio`) current target word is used as the decoder's next input rather than using the decoder's current guess.
- `gradient clipping` commonly used technique for countering the "exploding gradient" problem.

Sequence of operations :
1. Forward pass entire input batch through encoder
2. initialize decoder inputs as SOS_token and hidden state as the encoder's final hidden state
3. Forward input batch sequence through decoder one time step at a time
4. If teacher forcing : set next decoder input as the current target else : set next decoder input as the current decoder ouptput
5. Calculate and accumulate loss
6. Perform backpropagation
7. Clip gradients
8. Update encoder and decoder

In [ ]:

def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

Training iterations + save our model to run inferences or continue training !

In [ ]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

## Gready decoding for generating sentences

Decoding method when training is not using teacher forcing. At each time step we choose the word from decoder_output with the highest softmax value.

In [ ]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded word s to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

## Evaluation of model

In [ ]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            print("Quesion")
            print(input_sentence)
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            
            
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

## Run model

possible to laod from checkpoint

In [ ]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename ='data/save/cb_model/dailydialog/2-2_500/20000_checkpoint.tar'
#loadFilename ='data/save/RL_model_seq/dailydialog/15000_checkpoint.tar'

checkpoint_iter = 10000 # 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))
#print(loadFilename)

# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    #checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


# Kết quả của việc CB model

## Train !

In [ ]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 30000 #4000
print_every = 1
save_every = 1000

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

# Run training iterations
print("Starting Training!")
trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Streaming output truncated to the last 5000 lines.
Iteration: 25001; Percent complete: 83.3%; Average loss: 0.0795
Iteration: 25002; Percent complete: 83.3%; Average loss: 0.1402
Iteration: 25003; Percent complete: 83.3%; Average loss: 0.1663
Iteration: 25004; Percent complete: 83.3%; Average loss: 0.0794
Iteration: 25005; Percent complete: 83.4%; Average loss: 0.1038
Iteration: 25006; Percent complete: 83.4%; Average loss: 0.0893
Iteration: 25007; Percent complete: 83.4%; Average loss: 0.1438
Iteration: 25008; Percent complete: 83.4%; Average loss: 0.1357
Iteration: 25009; Percent complete: 83.4%; Average loss: 0.0935
Iteration: 25010; Percent complete: 83.4%; Average loss: 0.1245
Iteration: 25011; Percent complete: 83.4%; Average loss: 0.1360
Iteration: 25012; Percent complete: 83.4%; Average loss: 0.1170
Iteration: 25013; Percent complete: 83.4%; Average loss: 0.0856
Iteration: 25014; Percent complete: 83.4%; Average loss: 0.1183
Iteration: 25015; Percent complete: 83.4%; Average lo

## RUN AN PLAY

## Thử nghiệm câu đơn với SEQ2SEQ model

In [ ]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting
evaluateInput(encoder, decoder, searcher, voc)

> "I don't want to go home tonight",
Quesion
"I don't want to go home tonight",
Bot: why ? that s not so cute .
> "Shall we get started?",
Quesion
"Shall we get started?",
Bot: we don t have to make an appointment .
> "What's your favorite food?",
Quesion
"What's your favorite food?",
Bot: i like fish and vegetables . food
> "What kinds of films do you like?",
Quesion
"What kinds of films do you like?",
Bot: i like all kinds . about something like that .
> i m planning to be there at twelve .
Quesion
i m planning to be there at twelve .
Bot: we are going to do this week .
> "How long have you learning English?",
Quesion
"How long have you learning English?",
Bot: in english . most english in english . english
> "What's your favorite subject?",
Quesion
"What's your favorite subject?",
Bot: i d like to have her birthday and give you more sweet .
> "Do you need anything else?",
Quesion
"Do you need anything else?",
Bot: no . that s all . thank you . . .
> "Music is my hobby",
Quesion
"Mus

KeyboardInterrupt: ignored

## Reinforcement learning

### Forward and backward models 

In [ ]:
#forward_encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
#forward_decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
#forward_encoder = forward_encoder.to(device)
#forward_decoder = forward_decoder.to(device)

In [ ]:
forward_encoder = encoder
forward_decoder = decoder
forward_encoder = forward_encoder.to(device)
forward_decoder = forward_decoder.to(device)

In [ ]:
backward_encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
backward_decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
backward_encoder = backward_encoder.to(device)
backward_decoder = backward_decoder.to(device)

### Utility functions

In [ ]:
def convertResponse(response):
    size1 = len(response)
    size2 = batch_size
    npRes = np.zeros((size1, size2), dtype=np.int64)
    npLengths = np.zeros(size2, dtype=np.int64)
    for i in range(size1):
        prov = response[i].cpu().numpy()
        for j in range(prov.size):
            npLengths[j] = npLengths[j] + 1
            if prov.size > 1:
                npRes[i][j] = prov[j]
            else:
                npRes[i][j] = prov 
    res = torch.from_numpy(npRes)
    lengths = torch.from_numpy(npLengths)
    return res, lengths

In [ ]:
def convertTarget(target):
    size1 = len(target)
    size2 = batch_size
    npRes = np.zeros((size1, size2), dtype=np.int64)
    mask = np.zeros((size1, size2), dtype=np.bool_)
    npLengths = np.zeros(size2, dtype=np.int64)
    for i in range(size1):
        prov = target[i].cpu().numpy()
        for j in range(prov.size):
            npLengths[j] = npLengths[j] + 1
            if prov.size > 1:
                npRes[i][j] = prov[j]
            else:
                npRes[i][j] = prov 
                
            if npRes[i][j] > 0:
                mask[i][j] = True
            else:
                mask[i][j] = False
            
    res = torch.from_numpy(npRes)
    lengths = torch.from_numpy(npLengths)
    mask= torch.from_numpy(mask)
    max_target_len = torch.max(lengths) #.detach().numpy()
    return res, mask, max_target_len

In [ ]:
def transformTensorToSameShapeAs(tensor, shape):
    size1, size2 = shape
    npNewT = np.zeros((size1, size2), dtype=np.int64)
    npNewMask = np.zeros((size1, size2), dtype=np.bool_)
    tensorSize1, tensorSize2 = tensor.size()
    for i in range(tensorSize1):
        for j in range(tensorSize2):
            npNewT[i][j] = tensor[i][j]
            npNewMask[i][j]= True
    return torch.from_numpy(npNewT), torch.from_numpy(npNewMask)

### Training step for a single iteration 

In [ ]:
def RL(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, batch_size, teacher_forcing_ratio):
    # Set device options
    input_variable = input_variable.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)
    # Lengths for rnn packing should always be on the cpu
    lengths = lengths.to("cpu")
    
    #Initialize variables
    loss=0
    #print_losses = []
    response=[]
    
    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)
    
    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[SOS_token for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)
    
    
    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

     # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    
     # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            #print_losses.append(mask_loss.item() * nTotal)
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            #print_losses.append(mask_loss.item() * nTotal)
            
            #ni or decoder_output
            response.append(topi)
            
    return loss, max_target_len, response

Let's define the rewards method for each action.

It is composed of 3 different types of rewards :
- `ease of answering` which is basically saying that a turn generated by a machineshould be easy to respond to. 
- `information flow` : We  want  the  agent  to  contribute new information at each turn to keep the dialogue moving and avoid repetitive sequences. 
- `semantic coherence` measures the adequacy of responses to avoid situations in which the generated replies are highly rewarded but are ungrammatical or not coherent. 

In [ ]:
def easeOfAnswering(input_variable, lengths, dull_responses, mask, max_target_len, encoder, decoder, batch_size, teacher_forcing_ratio):
    NS=len(dull_responses)
    r1=0
    for d in dull_responses:
        d, mask, max_target_len = outputVar(d, voc)
        newD, newMask = transformTensorToSameShapeAs(d, input_variable.size())
        #tar, mask, max_target_len = convertTarget(d)
        forward_loss, forward_len, _ = RL(input_variable, lengths, newD, newMask, max_target_len, encoder, decoder, batch_size, teacher_forcing_ratio)
        # log (1/P(a|s)) = CE  --> log(P(a | s)) = - CE
        if forward_len > 0:
            r1 -= forward_loss / forward_len
    if len(dull_responses) > 0:
        r1 = r1 / NS
    return r1

In [ ]:
def informationFlow(responses):
    r2=0
    if(len(responses) > 2):
        #2 representations obtained from the encoder for two consecutive turns pi and pi+1
        h_pi = responses[-3]
        h_pi1 = responses[-1]
        # length of the two vector might not match
        min_length = min(len(h_pi), len(h_pi+1))
        h_pi = h_pi[:min_length]
        h_pi1 = h_pi1[:min_length]
        #cosine similarity 
        #cos_sim = 1 - distance.cosine(h_pi, h_pi1)
        cos_sim = 1 - distance.cdist(h_pi.cpu().numpy(), h_pi1.cpu().numpy(), 'cosine')
        #Handle negative cos_sim
        if np.any(cos_sim <= 0):
            r2 = - cos_sim
        else:
            r2 = - np.log(cos_sim)
        r2 = np.mean(r2)
    return r2

In [ ]:
def semanticCoherence(input_variable, lengths, target_variable, mask, max_target_len, forward_encoder, forward_decoder, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio):
    #print("IN R3:")
    #print("Input_variable :", input_variable.shape)
    #print("Lengths :", lengths.shape)
    #print("Target_variable :", target_variable.shape)
    #print("Mask :", mask.shape)
    #print("Max_Target_Len :", max_target_len)
    r3 = 0
    forward_loss, forward_len, _ = RL(input_variable, lengths, target_variable, mask, max_target_len, forward_encoder, forward_decoder, batch_size, teacher_forcing_ratio)
    ep_input, lengths_trans = convertResponse(target_variable)
    #print("ep_input :", ep_input.shape)
    #print("Lengths transformed :", lengths_trans.shape)
    ep_target, mask_trans, max_target_len_trans = convertTarget(input_variable)
    #print("ep_target :", ep_target.shape)
    #print("mask transformed :", mask_trans.shape)
    #print("max_target_len_trans :", max_target_len_trans)
    backward_loss, backward_len, _ = RL(ep_input, lengths_trans, ep_target, mask_trans, max_target_len_trans, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio)
    if forward_len > 0:
        r3 += forward_loss / forward_len
    if backward_len > 0:
        r3+= backward_loss / backward_len
    return r3

In [ ]:
l1=0.6
l2=0.1
l3=0.3


In [ ]:
def calculate_rewards(input_var, lengths, target_var, mask, max_target_len, forward_encoder, forward_decoder, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio):
    #rewards per episode
    ep_rewards = []
    #indice of current episode
    ep_num = 1
    #list of responses
    responses = []
    #input of current episode
    ep_input = input_var
    #target of current episode
    ep_target = target_var
    
    #ep_num bounded -> to redefine (MEDIUM POST)
    while (ep_num <= 10):
        
        print(ep_num)
        #generate current response with the forward model
        _, _, curr_response = RL(ep_input, lengths, ep_target, mask, max_target_len, forward_encoder, forward_decoder, batch_size, teacher_forcing_ratio)
        
        #Break if :
        # 1 -> dull response
        # 2 -> response is less than MIN_LENGTH
        # 3 -> repetition ie curr_response in responses
        if(len(curr_response) < MIN_COUNT):# or (curr_response in dull_responses) or (curr_response in responses)):
            break
            
            
        #We can add the response to responses list
        #curr_response = torch.LongTensor(curr_response).view(-1, 1)
        #transform curr_response size
        #target = torch.zeros(960, 1)
        #target[:15, :] = curr_response
        #curr_response = target
        #print(curr_response.size())
        #curr_response = torch.reshape(curr_response, (15, 64))
        #print(curr_response.size())
        #curr_response = curr_response.to(device)
        #responses.append(curr_response) 
        
        
        #Ease of answering
        r1 = easeOfAnswering(ep_input, lengths, dull_responses, mask, max_target_len, forward_encoder, forward_decoder, batch_size, teacher_forcing_ratio)
        
        #Information flow
        r2 = informationFlow(responses)
        
        #Semantic coherence
        r3 = semanticCoherence(ep_input, lengths, target_var, mask, max_target_len, forward_encoder, forward_decoder, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio)
        #Final reward as a weighted sum of rewards
        r = l1*r1 + l2*r2 + l3*r3
        print(f"reward:{r}")
      
        #Add the current reward to the list
        ep_rewards.append(r.detach().cpu().numpy())
        
        #We can add the response to responses list
        curr_response, lengths = convertResponse(curr_response)
        curr_response = curr_response.to(device)
        responses.append(curr_response)
        
        #Next input is the current response
        ep_input = curr_response
        #Next target -> dummy
        ep_target = torch.zeros(MAX_LENGTH,batch_size,dtype=torch.int64)
        #ep_target = torch.LongTensor(torch.LongTensor([0] * MAX_LENGTH)).view(-1, 1)
        ep_target = ep_target.to(device)
        
        #Turn off the teacher forcing  after first iteration -> dummy target
        teacher_forcing_ratio = 0
        ep_num +=1
    #Take the mean of the episodic rewards
    return np.mean(ep_rewards) if len(ep_rewards) > 0 else 0 

Dull responses 

In [ ]:
dull_responses = ["i do not know what you are talking about.", "i do not know.", 
 "you do not know.", "you know what i mean.", "i know what you mean.", 
 "you know what i am saying.", "you do not know anything."]
#dull_responses = []

In [ ]:
def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


### Training RL loop

In [ ]:
def trainingRLLoop(model_name, voc, pairs, batch_size, forward_encoder, forward_encoder_optimizer, forward_decoder, forward_decoder_optimizer, backward_encoder, backward_encoder_optimizer, backward_decoder, backward_decoder_optimizer,teacher_forcing_ratio, dull_responses, n_iteration, print_every, save_every, save_dir):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]
    plot_every=print_every
    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    plot_losses = []
    plot_loss=0
    #Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        print("Iteration", iteration)
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        ##MODIFS HERE
        # Zero gradients the optimizer
        forward_encoder_optimizer.zero_grad()
        forward_decoder_optimizer.zero_grad()
        
        backward_encoder_optimizer.zero_grad()
        backward_decoder_optimizer.zero_grad()
        
        #Forward
        forward_loss, forward_len, _ = RL(input_variable, lengths, target_variable, mask, max_target_len, forward_encoder, forward_decoder, batch_size, teacher_forcing_ratio)
        
        #Calculate reward
        reward = calculate_rewards(input_variable, lengths, target_variable, mask, max_target_len, forward_encoder, forward_decoder, backward_encoder, backward_decoder, batch_size, teacher_forcing_ratio)
        print(reward)
        reward_arr.append(reward)
        #Update forward seq2seq with loss scaled by reward
        loss = forward_loss * reward
        
        loss.backward()
        forward_encoder_optimizer.step()
        forward_decoder_optimizer.step()

        # Run a training iteration with batch
        print_loss += loss / forward_len
        plot_loss += loss / forward_len
        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0
        if iteration % plot_every == 0:
            plot_loss_avg = plot_loss / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss= 0
            
        #SAVE CHECKPOINT TO DO
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name)#, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': forward_encoder.state_dict(),
                'de': forward_decoder.state_dict(),
                'en_opt': forward_encoder_optimizer.state_dict(),
                'de_opt': forward_decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc.__dict__,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))
    showPlot(plot_losses)

In [ ]:
def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
reward_arr=[]

## RUN 

In [ ]:
#Configure RL model

model_name='RL_model_seq'
n_iteration = 10000
print_every=500
save_every=1000
learning_rate = 0.0001
decoder_learning_ratio = 5.0
teacher_forcing_ratio = 0.5

# Ensure dropout layers are in train mode
forward_encoder.train()
forward_decoder.train()

backward_encoder.train()
backward_decoder.train()

# Initialize optimizers
print('Building optimizers ...')
forward_encoder_optimizer = optim.Adam(forward_encoder.parameters(), lr=learning_rate)
forward_decoder_optimizer = optim.Adam(forward_decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
backward_encoder_optimizer = optim.Adam(backward_encoder.parameters(), lr=learning_rate)
backward_decoder_optimizer = optim.Adam(backward_decoder.parameters(), lr=learning_rate * decoder_learning_ratio)

# If you have cuda, configure cuda to call
for state in forward_encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in forward_decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()
            
for state in backward_encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in backward_decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()
        
# Run training iterations
print("Starting Training!")
trainingRLLoop(model_name, voc, pairs, batch_size, forward_encoder, forward_encoder_optimizer, forward_decoder, forward_decoder_optimizer, backward_encoder, backward_encoder_optimizer, backward_decoder, backward_decoder_optimizer, teacher_forcing_ratio, dull_responses, n_iteration, print_every, save_every, save_dir)

NameError: ignored

In [ ]:
print(len(reward_arr))

352


In [ ]:
import pandas as pd
df = pd.DataFrame({'reward':reward_arr})
print (df)

       reward
0    0.000000
1   -5.658504
2   -5.606884
3   -5.749689
4    0.000000
..        ...
347       NaN
348  0.000000
349  0.000000
350  0.000000
351  0.000000

[352 rows x 1 columns]


In [ ]:
%cd /content/
df.to_csv('0.6_0.1_0.3.csv')#ghi file csv và tải về

/content


In [ ]:
%cd /content/drive/MyDrive/CS106 # trở lại chỗ load model

/content/drive/MyDrive/CS106


# Load model trong drive ra để sử dụng

In [ ]:
loadFilename ='data/save/RL_model_seq0.2-0.2-0.6/dailydialog/10000_checkpoint.tar' # thay đổi model, và trọng số ở đây 'data/save/XXX/dailydialog/YYY_checkpoint.tar' với XXX là tên model RL chọn,YYY là số iteration
if loadFilename:
    # If loading on same machine the model was trained on
    #checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
forward_encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
forward_decoder= LuongAttnDecoderRNN(attn_model, embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
if loadFilename:
    forward_encoder.load_state_dict(encoder_sd)
    forward_decoder.load_state_dict(decoder_sd)
# Use appropriate device
forward_encoder = forward_encoder.to(device)
forward_decoder = forward_decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [ ]:
def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            print("Quesion")
            print(input_sentence)
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            
            
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [ ]:
def Simulate(forward_encoder, forward_decoder, searcher, voc,input_sentence):
  #print(input_sentence)
  for i in range(0,8):
    try:
              # Get input sentence
              # Check if it is quit case
              if input_sentence == 'q' or input_sentence == 'quit': break
              # Normalize sentence
              
              input_sentence = normalizeString(input_sentence)
              # Evaluate sentence
              
              
              output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
              # Format and print response sentence
              output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
              print( ' '.join(output_words))
          
              input_sentence=' '.join(output_words)

    except KeyError:
              print("Error: Encountered unknown word.")

## Chạy thử câu đơn với model RL

In [ ]:
# Set dropout layers to eval mode
forward_encoder.eval()
forward_decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(forward_encoder, forward_decoder)

# Begin chatting
evaluateInput(forward_encoder, forward_decoder, searcher, voc)

KeyboardInterrupt: ignored

In [ ]:
def EvaluateMultiInput(forward_encoder, forward_decoder, searcher, voc,input_arr):
  for input_sentence in input_arr:
    try:
              # print(input_sentence)
              # Get input sentence
              # Check if it is quit case
              if input_sentence == 'q' or input_sentence == 'quit': break
              # Normalize sentence
              
              input_sentence = normalizeString(input_sentence)
              # Evaluate sentence
              
              
              output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
              # Format and print response sentence
              output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
              print( ' '.join(output_words))
          
              input_sentence=' '.join(output_words)

    except KeyError:
              print("Error: Encountered unknown word.")


## Đánh giá nhiều câu cùng lúc

In [ ]:
# Set dropout layers to eval mode
forward_encoder.eval()
forward_decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(forward_encoder, forward_decoder)
input_arr=[
"I don't want to go home tonight",
"Shall we get started?",
"How come you never say it?",
"Let's try other places",
"How about tea?",
"What a lovely day we are having",
"Nice to meet you",
"Would you like to have a chat?",
"What made you think so?",
"I need you to stay here",
"Can you repeat that?",
"I am a human",
"The food was delicious",
"I am honored to be here",
"I go to a university",
"Why don't we go out for dinner?",
"Are you sure about that?",
"Can you be more specific?",
"I don't think so",
"I absolutely agree",
"Music is my hobby",
"I brought this since you like it",
"Do you agree with that?",
"Congratulations",
"I am happy for you",
"I am a women",
"He is handsome, don't you think?",
"She is beautiful, wouldn't you agree?",
"Likewise, I share the same thoughts",
"This is nothing to be scared of",
"A phone call is a good idea",
"Bring me a cup of water, please",
"That is horrible",
"I can't imagine what you are going through",
"I have to say no",
"Not at all, i don't mind",
"That has bothered me for a while",
"But how would that change anything?",
"Why bother with it?",
"Let's change topic",
"Please, it is my treat",
"You are beautiful",
"Can I have a bit of your time?",
"How would you go about doing that?",
"Can it be done?",
"You mustn't do it",
"Can you tell?",
"How old are you?",
"What is your full name?",
"Do you have any feelings for me?",
"How much time do you have here?",
"Do you play football?",
"Where are you from?",
"How is the weather today?",
"Did you go to school?",
"How is your family doing?",
"Where are you going?",
"Where do you live?",
"When is your birthday?",
"What is your phone number?",
"What's your email address?",
"What do you do?",
"What's your job?",
"What line of work are you in?",
"What company do you work for?",
"Are you married?",
"Are you in a relationship?",
"Do you have any siblings?",
"Who do you live with?",
"What time do you get up?",
"What time do you have breakfast?",
"What time do you go to bed?",
"What's your hobby?",
"What's your favorite food?",
"What's your favorite color?",
"What's your favorite drink?",
"What kinds of films do you like?",
"Did you like the movie?",
"Where do you study?",
"What time do you go to school?",
"How do you get to school?",
"What is your major?",
"Why do you study English?",
"Why are you studying English?",
"What do you want to do after you graduate?",
"How did you learn English?",
"How many languages do you speak?",
"Can you speak English?",
"How long have you learning English?",
"Which grade are you in?",
"Which year are you in?",
"Do you have any exams coming up?",
"What's your favorite subject?",
"What's the weather like?",
"How's the weather?",
"What's the temperature?",
"What time is it?",
"What's the date today?",
"How are you?",
"Where are you going?",
"What are you going to do today?",
"What are you doing?",
"Where would you like to go?",
"What's the matter?",
"Is there anything I can do to help?",
"What's on your mind?",
"What did you do last night?",
"What are you going to do tomorrow?",
"What sports can you play?",
"Can you give me a hand?",
"Could you do me a favor?",
"Could you please give me that book?",
"Would you mind opening the window?",
"How can I help you?",
"May I help you?",
"Can I try it on?",
"Can I try it on somewhere?",
"What size do you wear?",
"What size do you take?",
"Is that a good fit?",
"Have you got something bigger?",
"How much is it?",
"How much does it cost?",
"How would you like to pay?",
"Can I pay by credit card?",
"Do you need anything else?",
"How are you feeling?",
"How was the party?",
"Are you ready to order?",
"Would you like chicken or pasta?",
"What would you like to drink?",
"Did you save room for dessert?",
"How does it taste?",
"Can I get you anything else?",
"What do you think about this?",
"Are you here with anybody?",
"Have you got a pet?",
"How do you do?",
"What's this?",
"What does she look like?",
"How is she?",
"What is he like?",
"Are you hungry?",
"How do you feel about him?",
"May I open the window?",
"Do you need help?",
"What do you do every day?",
"What are you planning to do today?",
"What are you planning for after work?",
"Are you free tomorrow?",
"Will you join me for coffee?",
"Could we have lunch together one day?",
"Would you like to have dinner with me?",
"Where do you want to go?",
"Would you like to come to the cinema with me tonight?",
"Would you like to play a round of golf this weekend?",
"How about coming to the barbecue at the tennis club?",
"Have you been waiting long?",
"How do you get to work?",
"Is it close to the subway station?",
"How long does it take?",
"How often do you ride the bus?",
"Could you tell me how to get to the police station?",
"Excuse me! Is there a bank near here?",
"What's the best way to the museum?",
"Can I speak to her, please?",
"When will she be back?",
"Did you get my message?",
"Would you like to leave a message?",
"How long have you been working here?",
"What time does the meeting start?",
"What time does the meeting finish?",
"Are you sick?",
"What are your symptoms?",
"How long have you been feeling like this?",
"Are you on any sort of medication?",
"Where's the counter, please?",
"May I have your passport, please?",
"Do you have anything to declare?",
"Do you have any bags to check?",
"Would you prefer an aisle seat or a window seat?",
"What's the purpose of your trip?",
"Where are you going to be staying?",
"How long are you going to stay here?",
"Could you tell me where Gate E is?",
"Can I help you find something?",
"Could you tell me where the meat is?",
"Where can I find the non-fat biscuits?",
"How much would you like?",
"Can you offer me any discount?",
"Do you need any help packing?",
"Would you like a receipt?",
"What times are you open?",
"What time do you open?",
"What time do you close?",
"Do you have a reservation?",
"What kind of room would you like?",
"May I see your ID, please?",
"Could I see the room?",
"Do you have a credit card, sir?",
"How long will you be staying?",
"Do you have any rooms available for the 14th of April?",
"How many places have you traveled to?",
"Have you ever been abroad?",
"What do you usually do during your trip?",
"Do you prefer traveling by car, train or plane?",
"Do you prefer traveling alone or joining a guided tour?",
"Did you enjoy yourself last night?",
"I'm ready to try again, if you are?",
"It looks like you're in trouble there. Can I help?",
"You got a package for me?",
"Is there something you want to tell me, son?",
"How, exactly, did you think that having an affair would help our marriage?",
"You blame me but where were you at the time?",
"What's in that bag and why are you hiding it here?",
"She's been missing since Friday and you're not worried?",
"Could you be happy here with me?",
"What do you remember about your mother?",
"Do you think he crashed the car on purpose?",
"Do you ever think we should just stop doing this?",
"Have you read the newspaper stories about my wife?",
"You had time to call the police. Why didn't you?",
"Are you taking his side against me?",
"Do you remember we used to do that in school?",
"How did he get away?",
"You've taken her back? You can't be serious?",
"Spare some change, please?",
"Were you really looking for me?",
"What did you have in the garden?",
"Why did you talk like that?",
"What do you mean, you've lost the lottery ticket?",
"Am I supposed to be scared now?",
"I just want a nice, easy life. What's wrong with that?",
"How have you been all these years?",
"Oh yeah, you really told him, didn't you?",
"Why didn't he come and talk to me himself?",
"Why would you want to put yourself through something like that?",
"Perhaps you'll take me out one day - or do I have to make an appointment?",
"Where would a child go in a small place like this?",
"You don't think that was just lemonade in your glass, do you?",
"How could I leave my children?",
"Would you come to my funeral?",
"Does he know about the baby?",
"Am I under arrest, or not?",
"How long have you been standing there?",
"What are you major in university",
]
EvaluateMultiInput(forward_encoder, forward_decoder, searcher, voc,input_arr)

so why don t you ?
we have we put out the top we we . .
i saw a three times if i ve looked for a salesperson .
oh . i ll tell be . .
only about tea tea . i ll have tea at .
we d like to order breakfast a may and we .
would you like to show me a meeting ?
what ll i do ? fine ? fine ?
she does always good for me . . plus .
that s my pleasure . have
yes i have . so . camera .
which one would you be ? moment
before the food really is the food . food food .
i have been going to have your flight .
what did you do ? ? ?
great idea . i know where you want .
i m not certain . i ve always loved such such a few .
i want to got my things . my for my my my . .
that s a good idea . . concert .
do you have your decision with the suspect ? ?
what kind of music do you you you ?
i love the same day .
yes i have . your place is very much of this use .
thanks . i appreciate it . . .
thank you for the same . .
i d like to see your please . . ?
yes . let s say . said . . .
no . it s very kind of you .
that 

## Tạo đoạn hội thoại ngẫu nhiên

In [ ]:
# Set dropout layers to eval mode
forward_encoder.eval()
forward_decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(forward_encoder, forward_decoder)
#input_sentence=input('')
input_sentence=input('')
Simulate(forward_encoder,forward_decoder,searcher,voc,input_sentence)

hi
oh sounds good . i ll have to meet you at .
thank you for coming . and you re coming along .
i m just coming coming . what about you ? ?
i m just at work house . . . .
what are you going to do then ? do do !
can t i sit just put it on the ? put in the
i hope you ll do the wrong to wash . . .
all right . ?


In [ ]:
# Set dropout layers to eval mode
forward_encoder.eval()
forward_decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(forward_encoder, forward_decoder)
questions = [
    "Hi",
    "Hello",
    "Nice to see you",
    "How are you doing?",
    "Good morning",
    "Good evening",
    "What is your name?",
    "How can I help you?",
    "What are you up to?",
    "Do you want to hang out?",
    "Let's go out for dinner",
    "I will see you again soon",
    "What do you want to talk about?",
    "Can you help me?",
    "When are you free?",
    "How are you at school?"
]

for question in questions:
  print(question)
  Simulate(forward_encoder,forward_decoder,searcher,voc,question)
  print("-------------------------------------------")

Hi
oh that s a good . see you there .
thank you . . you . . . .
not ! . . . .
what s you matter ?
i m not a . you in what your .
well that sounds good . i will always go in the .
really ? i love you !
you are so right . i you you to
-------------------------------------------
Hello
hi . ?
no . . . what s happening ?
that s a night . . . . . . . . .
yeah . the performance the the
let s show it .
do we get something wrong ?
it s up . no because t . .
what ?
-------------------------------------------
Nice to see you
it s a good time hasn t you ? family family
yes i can do very much . family .
are you willing to go go to school by
which school sounds good ?
i think most is the most .
the best of the is the the . .
it goes like . i ll take it .
how much is it ?
-------------------------------------------
How are you doing?
i m fine . thanks for asking .
what did you need ?
i just need some hair .
how much did you get me ?
it s the end of the . . .
what does the dear say ?
it s to the .
do 